# 전체 구조 요약
1. 문서 로드 (JSON, TXT 등)
2. 청크 분할 (TextSplitter)
3. 임베딩 (Upstage Embedding 사용) + 벡터 저장소 (FAISS)
4. 질문 입력 → 유사 문서 검색
5. LLM에게 prompt 생성 → 답변 출력

## 1. 문서 로드
### json 경로 설정

In [4]:
import os


# 경로 설정
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "../EDA/data")
data_path = os.path.join(DATA_DIR, "combined_service_data_merged.json")
prev_path = os.path.join(DATA_DIR, "combined_service_data_merged_prev.json")

In [5]:
import json
from pathlib import Path

# JSON 로드 및 개수 출력
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"📦 전체 서비스 개수: {len(data)}개")


📦 전체 서비스 개수: 8850개


### json 파일 불러와서 langchain 문서로 변환

In [6]:
import json
import shutil
from langchain.docstore.document import Document

# 🔹 JSON 로드 (new)
with open(data_path, "r", encoding="utf-8") as f:
    new_data = json.load(f)
new_dict = {item["서비스ID"]: item for item in new_data}

# 🔹 JSON 로드 (prev)
if os.path.exists(prev_path):
    with open(prev_path, "r", encoding="utf-8") as f:
        prev_data = json.load(f)
    prev_dict = {item["서비스ID"]: item for item in prev_data}
else:
    prev_data = []
    prev_dict = {}

    
# 변경 감지
added, updated, deleted = [], [], []

# new_dict의 모든 서비스ID(sid)와 해당 데이터(item)에 대해 반복
for sid, item in new_dict.items():
    # 이전 데이터(prev_dict)에 현재 서비스ID가 없으면 → 신규 추가된 항목으로 간주
    if sid not in prev_dict:
        added.append(item)  # 추가 목록에 포함
    # 이전에도 존재했지만 내용이 다른 경우 → 수정된 항목으로 간주
    elif json.dumps(item, sort_keys=True, ensure_ascii=False) != json.dumps(prev_dict[sid], sort_keys=True, ensure_ascii=False):
        updated.append(item)  # 수정 목록에 포함

# 이전 데이터(prev_dict)에만 있고 현재 데이터(new_dict)에는 없는 서비스ID를 찾음
for sid in prev_dict:
    # 현재 데이터에 존재하지 않으면 → 삭제된 것으로 간주
    if sid not in new_dict:
        deleted.append(prev_dict[sid])  # 삭제 목록에 포함


print(f"🆕 추가된 문서: {len(added)}개")
print(f"🔄 수정된 문서: {len(updated)}개")
print(f"❌ 삭제된 문서: {len(deleted)}개")

# ✅ 임베딩 대상: 추가 + 수정 문서만
target_docs = added + updated

# LangChain 문서 변환
documents = []
for item in target_docs:
    조건_태그 = [k for k, v in item.get("조건", {}).items() if v]

    content = f"""
서비스명: {item.get('서비스명')}
서비스목적: {item.get('서비스목적')}
지원대상: {item.get('지원대상')}
지원내용: {item.get('지원내용')}
신청방법: {item.get('신청방법')}
신청기한: {item.get('신청기한')}
선정기준: {item.get('선정기준')}
구비서류: {item.get('구비서류')}
소관기관: {item.get('소관기관명')}
문의처: {item.get('문의처')}
온라인신청URL: {item.get('온라인신청사이트URL')}
법령: {item.get('법령')}
해당 조건: {', '.join(조건_태그)}
"""

    doc = Document(
        page_content=content.strip(),
        metadata={
            "서비스ID": item.get("서비스ID"),
            "서비스명": item.get("서비스명"),
            "조건": item.get("조건", {}),
            "원본": item
        }
    )
    documents.append(doc)

print(f"✅ LangChain 문서 생성 완료: {len(documents)}개")
if documents:
    print("🔍 예시 문서 내용:\n", documents[0].page_content[:3000])

🆕 추가된 문서: 0개
🔄 수정된 문서: 0개
❌ 삭제된 문서: 0개
✅ LangChain 문서 생성 완료: 0개


In [7]:
print("✅ new_data 개수:", len(new_data))
print("✅ prev_data 개수:", len(prev_data))

✅ new_data 개수: 8850
✅ prev_data 개수: 8850


## 2. 청크 분할
문자수 기반과 토큰 기반으로 나뉘는데 둘 중 하나만 돌릴 것.
문장 단위 의미 보존이 더 중요하다 판단되면 문자수 기반 / 비용 정확도, 입력 제한 관리 중요하다 판단되면 토큰 기반

### 문자수 기반 청크 분할
chunk_size=500 → 청크 하나의 page_content 길이가 최대 500자로 제한됨

chunk_overlap=100 → 이전 청크 끝 100자 → 다음 청크 시작에 겹쳐서 들어감

Recursive → 줄바꿈(\n) → 띄어쓰기( ) → 아무 데서나 ("") 순서로 분할 시도
→ 가능한 한 문장 단위 또는 문단 단위로 끊으려 노력함

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 문자 기반 텍스트 분할기 import

# 문자 수 기반으로 텍스트 청크 분할기 정의
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # 각 청크의 최대 문자 수
    chunk_overlap=100     # 청크 간 중복 문자 수 (문맥 유지를 위해 사용)
)

# 기존에 만든 LangChain 문서 리스트 (`documents`)에 대해 분할 수행
split_docs = text_splitter.split_documents(documents)  # 각 Document의 page_content를 기준으로 분할

# 결과 확인
if split_docs:
    print(f"✅ 전체 청크 수: {len(split_docs)}")
    print("🔍 첫 번째 청크 예시:\n", split_docs[0].page_content)
else:
    print("⚠️ 문서 내용에 변화가 없어 청크가 생성되지 않았습니다.")


⚠️ 문서 내용에 변화가 없어 청크가 생성되지 않았습니다.


### 토큰 기반 청크 분할
chunk_size: 각 문서를 몇 토큰 단위로 나눌지 결정하는 가장 중요한 파라미터.

chunk_overlap: 다음 청크가 이전 청크의 마지막 일부를 포함하게 해서 문맥을 자연스럽게 이어주기 위한 장치.

cl100k_base: OpenAI의 최신 모델에서 사용하는 토크나이저. Solar의 임베딩과 토큰 구조가 유사하다고 알려져 있음.

In [9]:
from langchain.text_splitter import TokenTextSplitter  # 토큰 기반 텍스트 분할기
from langchain_openai import OpenAIEmbeddings  # 임베딩 모델 (토크나이저 공유)

# OpenAI 기준: text-embedding-ada-002는 max 8191 tokens
# 안전하게 512 ~ 1024 단위로 청크 나누는 게 일반적
token_splitter = TokenTextSplitter(
    chunk_size=1000,        # 한 청크당 최대 토큰 수
    chunk_overlap=150       # 중복 토큰 수 (문맥 유지용) chunk_overlap 기술 적용
)

# 문서 리스트 분할
token_split_docs = token_splitter.split_documents(documents)

# 결과 확인
if token_split_docs:
    print(f"✅ 토큰 기반 청크 수: {len(token_split_docs)}")
    print("🔍 예시 청크:\n", token_split_docs[0].page_content)
else:
    print("⚠️ 문서 내용에 변화가 없어 토큰 청크가 생성되지 않았습니다.")

⚠️ 문서 내용에 변화가 없어 토큰 청크가 생성되지 않았습니다.


### Solar Embedding 비용 계산 코드

In [10]:
import tiktoken  # 토큰 계산용
from tqdm import tqdm

# 1. 토크나이저 로드 (Solar는 cl100k_base 계열과 거의 동일)
tokenizer = tiktoken.get_encoding("cl100k_base")

# 2. 전체 토큰 수 계산
total_tokens = 0
for doc in tqdm(token_split_docs, desc="🔍 토큰 수 계산 중"):       # 계산하고 싶은 청크
    tokens = tokenizer.encode(doc.page_content)
    total_tokens += len(tokens)

# 3. Solar 실제 요율 기준 단가 설정 (1,000 tokens = $0.0000973)
usd_per_1000_tokens = 0.0000973
estimated_cost_usd = (total_tokens / 1000) * usd_per_1000_tokens

# 4. 한화 환율로 변환 (예: 1 USD = 1350원 가정)
exchange_rate = 1350
estimated_cost_krw = estimated_cost_usd * exchange_rate

# 5. 결과 출력
print(f"\n📊 총 청크 수: {len(token_split_docs)}")
print(f"🔢 총 토큰 수: {total_tokens:,} tokens")
print(f"💵 예상 비용: ${estimated_cost_usd:.4f} USD")
print(f"💰 예상 비용: 약 {estimated_cost_krw:,.0f}원 (환율: {exchange_rate}원/USD)")


🔍 토큰 수 계산 중: 0it [00:00, ?it/s]


📊 총 청크 수: 0
🔢 총 토큰 수: 0 tokens
💵 예상 비용: $0.0000 USD
💰 예상 비용: 약 0원 (환율: 1350원/USD)


## 3. 임베딩 (Upstage Embedding 사용) + 벡터 저장소 (FAISS)
### API 가져오기

In [11]:
from dotenv import load_dotenv
import os

# ✅ RAG/.env 경로를 직접 지정
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # 현재 디렉토리의 부모(RAG)
env_path = os.path.join(base_dir, ".env")  # RAG/.env

load_dotenv(dotenv_path=env_path)

# ✅ 환경변수 불러오기
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
UPSTAGE_API_URL = os.getenv("UPSTAGE_API_URL")

# ✅ 결과 확인
print("🔐 API KEY:", "불러옴" if UPSTAGE_API_KEY else "❌ 없음")
print("🌐 API URL:", UPSTAGE_API_URL if UPSTAGE_API_URL else "❌ 없음")
print("📁 base_dir:", base_dir)


🔐 API KEY: 불러옴
🌐 API URL: https://api.upstage.ai/v1/embeddings
📁 base_dir: c:\Users\jihu6\code\RAG


### SOLAR API로 임베딩 + FAISS에 벡터 저장

In [12]:
from tqdm import tqdm  # 진행률 표시용 라이브러리
import time  # 요청 간 간격 조정용 (선택)
import requests
from langchain_core.embeddings import Embeddings
from typing import List

class UpstageEmbeddings(Embeddings):
    def __init__(self, api_key: str, api_url: str, batch_size: int = 64):
        self.api_key = api_key  # API 키 저장
        self.api_url = api_url  # 요청 보낼 URL 저장
        self.batch_size = batch_size  # 한 번에 보낼 텍스트 개수

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        all_embeddings = []

        for i in tqdm(range(0, len(texts), self.batch_size), desc="🔄 Solar 임베딩 중"):
            batch = texts[i:i + self.batch_size]

            # 👇 디버깅용: 문자열 아닌 항목 검사
            if not all(isinstance(t, str) for t in batch):
                print("❗문자열이 아닌 데이터가 포함됨:")
                for t in batch:
                    print(type(t), "→", repr(t)[:100])

            # ✅ Solar API에 필요한 model 필드 추가!
            payload = {
                "input": batch,
                "model": "embedding-query"  # 반드시 포함되어야 함
            }

            response = requests.post(self.api_url, headers=headers, json=payload)

            # 에러 출력
            if response.status_code != 200:
                print("❌ 에러 응답 상태:", response.status_code)
                print("❌ 에러 응답 본문:", response.text)

            response.raise_for_status()
            result = response.json()
            all_embeddings.extend([item["embedding"] for item in result["data"]])
            time.sleep(0.1)  # 요청 너무 빠르게 보내지 않도록 딜레이

        return all_embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]


In [13]:
# 🔹 FAISS 및 임베딩 클래스 임포트
from langchain.vectorstores import FAISS

# 🔹 Upstage 임베딩 객체 생성 (앞서 만든 사용자 정의 클래스 사용)
embedding = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,  # .env에서 불러온 API 키
    api_url=UPSTAGE_API_URL,  # .env에서 불러온 API URL
    batch_size=64             # 한 번에 64개씩 처리
)

# 🔹 청크가 존재할 경우에만 FAISS 인덱스 생성 및 저장
if token_split_docs:
    db = FAISS.from_documents(token_split_docs, embedding)  # 문서 임베딩 후 FAISS 인덱스 생성
    db.save_local("faiss_index")  # 인덱스를 로컬 디렉토리에 저장
    print("✅ FAISS 인덱스 저장 완료!")
else:
    print("⚠️ 임베딩할 문서가 없어 FAISS 인덱스를 생성하지 않습니다.")

⚠️ 임베딩할 문서가 없어 FAISS 인덱스를 생성하지 않습니다.


## 4. 질문 입력 → 유사 문서 검색

### 벡터 데이터 베이스 로딩

In [14]:
# ✅ allow_dangerous_deserialization=True 추가
db = FAISS.load_local(
    "faiss_index",
    embeddings=embedding,
    allow_dangerous_deserialization=True  # ✅ Pickle 로드 허용
)

### 검색할 질문 입력 & 5 회까지 채팅 기억
Ram에만 저장되도록 설정

In [ ]:
# 검색할 질문 예시
query = "청년이 받을 수 있는 주거 지원은 뭐가 있지?"

### 유사 문서 검색

In [16]:
# 유사 문서 검색 (상위 3개)
docs = db.similarity_search(query, k=3)

🔄 Solar 임베딩 중: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


#### 유사 문서 검색 결과 확인

In [ ]:
# 🔍 유사 문서 + 조건 출력
for i, doc in enumerate(docs):
    print(f"📄 문서 {i+1}")
    
    # 🔸 문서 요약 (앞부분만 출력)
    print(doc.page_content[:500])
    
    # 🔸 조건 정보 출력
    조건들 = doc.metadata.get("조건", {})
    선택된_조건 = [k for k, v in 조건들.items() if v]
    
    print(f"📌 조건 태그: {', '.join(선택된_조건) if 선택된_조건 else '조건 없음'}")
    print("-" * 50)

# ✅ 검색된 문서 내용 합치기
retrieved_text = "\n\n".join([doc.page_content for doc in docs])

📄 문서 1
서비스명: 서울시 청년 월세 지원
서비스목적: 청년층의 주거비 부담 완화를 통한 청년 주거 수준 향상으로 사회 진입을 돕고 생애 다음단계(내 집 마련, 결혼, 출산, 양육)로의 성장 지원
지원대상: ○  서울 월세 거주 19세 ~ 39세 이하 무주택 청년 1인 가구 (청년인 동거인 및 형제자매 있는 경우 신청 가능)
    - 소득 기준 : 기준 중위소득 150% 이하 
    - 재산 기준 : 일반재산 1억 3천만원 이하, 자동차 2,500만원 미만
    - 거주 요건 : 임차 보증금 8천만원 이하, 월세 60만원 이하 건물 월세 거주
지원내용: ○ 사업 내용 : 최대 12개월 간 월 20만원 범위에서 월세 지원
○ 소요재원 : 시비 100%
신청방법: 서울주거포털 : https://housing.seoul.go.kr 청년월세지원란에 온라인 신청,접수
신청기한: 2025.6월(예정)
선정기준: None
구비서류: 확정일자가 날인된 임대차계약서 사본 1부, 월세이체 증빙서류
📌 조건 태그: 남성, 여성, 중위소득 0~50%, 중위소득 51~75%, 중위소득 76~100%, 중위소득 101~200%, 중위소득 200% 초과, 예비부모/난임, 임산부, 출산/입양, 농업인, 어업인, 축산업인, 임업인, 초등학생, 중학생, 고등학생, 대학생/대학원생, 해당사항없음, 근로자/직장인, 구직자/실업자, 장애인, 국가보훈대상자, 질병/질환자, 1인가구
--------------------------------------------------
📄 문서 2
서비스명: 울산 청년가구 주거비 지원사업 지원 신청
서비스목적: 임차료 등 주거비 지원으로 청년층의 주거안정 및 지역정착으로 결혼율 제고
지원대상: - 19세~39세 무주택 미혼 청년가구 세대주 
- 임차보증금 1억원이하 및 월세 50만원 이하인 주택 거주  
- 기준 중위소득 150% 이하(건강보험료 납부 기준)
지원내용: 월 임차료비용 10만원 실비지급
월 임차보증금이자비용 5만원 실비지급
※ 주거급여 수급

## 5. LLM에게 prompt 생성 → 답변 출력

### PROMPT 생성

In [29]:
retrieved_text = "\n\n".join([doc.page_content for doc in docs])

prompt = f"""
You are an assistant for answering questions about Korean government support policies.
Use the following retrieved context to answer the user's question.
If the answer is not in the context, say "잘 모르겠습니다."
Respond in Korean.

# Context:
{retrieved_text}

# Question:
{query}

# Answer:
"""

### 결과 출력

In [30]:

# ✅ Solar Chat API 호출
url = "https://api.upstage.ai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {UPSTAGE_API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "model": "solar-pro",
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.7,
    "max_tokens": 1024,
    "stream": False
}

response = requests.post(url, headers=headers, json=payload)
response.raise_for_status()

# ✅ 결과 출력
result = response.json()
print("🤖 Solar Chat 응답:")
print(result["choices"][0]["message"]["content"])

🤖 Solar Chat 응답:
청년이 받을 수 있는 주거 지원으로는 서울시 청년 월세 지원, 울산 청년가구 주거비 지원사업 지원이 있습니다. 서울시 청년 월세 지원은 서울시 청년을 대상으로 하며, 최대 12개월 간 월 20만원 범위에서 월세를 지원합니다. 울산 청년가구 주거비 지원사업은 19세~39세 무주택 미혼 청년가구를 대상으로 하며, 월 임차료비용 10만원과 월 임차보증금이자비용 5만원을 지원합니다.
